https://haystack.deepset.ai/tutorials/40_building_chat_application_with_function_calling

components: 
InMemoryDocumentStore, SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder, InMemoryEmbeddingRetriever, ChatPromptBuilder, OpenAIChatGenerator, ToolInvoker

OpenAPI API key

In [1]:
import os
openai_key = False
hf_token = False 
with open("secrets") as file:
    for line in file.readlines():
        key,value = line.strip().split("=")
        if key == 'OPENAI_API_KEY':
            openai_key = True
            os.environ[key]=value
        elif key == 'HF_TOKEN':
            hf_token = True
            os.environ[key]=value
assert openai_key, 'OPENAI_API_KEY not found'
assert hf_token, 'HF_TOKEN not found'

# assert hf_token, 'HF_TOKEN not found'
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
#setup OPENAI_API_KEY
from pprint import pprint

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

2025-01-31 09:44:23.865609: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 09:44:23.873153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-31 09:44:23.881909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-31 09:44:23.884535: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 09:44:23.891184: I tensorflow/core/platform/cpu_feature_guar

In [2]:
messages = [
    ChatMessage.from_system('Always respond in German'),
    ChatMessage.from_user('Briefly explain about the current state of health care in the US'),
]

chat_gen = OpenAIChatGenerator(model="gpt-4o-mini")
# chat_gen = OpenAIChatGenerator(model="gpt-4o-mini", streaming_callback = callback_fn)
res = chat_gen.run(messages=messages)

# document_store = InMemoryDocumentStore(embedding_similarity_function='cosine')
# text_embeder = SentenceTransformersTextEmbedder()
# retriever = InMemoryEmbeddingRetriever(document_store=document_store)

In [3]:
print(res['replies'][0]._content[0].text)

Das Gesundheitswesen in den USA steht vor mehreren Herausforderungen. Obwohl das Land über einige der fortschrittlichsten medizinischen Einrichtungen und Technologien verfügt, gibt es bedeutende Probleme wie hohe Kosten, ungleiche Zugänglichkeit und eine große Anzahl von Menschen ohne Krankenversicherung. Der Affordable Care Act hat zwar dazu beigetragen, die Zahl der Unversicherten zu reduzieren, viele Amerikaner kämpfen jedoch weiterhin mit hohen Prämien und Selbstbeteiligungen. Zudem gibt es Diskussionen über die Notwendigkeit von Reformen, um die Qualität der Versorgung zu verbessern und die Kosten zu senken. Telemedizin und digitale Gesundheit nehmen an Bedeutung zu, insbesondere seit der COVID-19-Pandemie.


In [4]:
facts = [
    "The Earth is the only planet in our solar system not named after a god.",
    "The Amazon rainforest produces more than 20% of the world's oxygen supply.",
    "Antarctica is the driest, windiest, and coldest continent.",
    "There are more than 24 time zones around the world.",
    "The Great Wall of China is the longest man-made structure in the world.",
    "Mount Everest is the highest point on Earth.",
    "The Pacific Ocean is the largest and deepest ocean on Earth.",
    "Russia is the largest country by land area.",
    "The Sahara Desert is the largest hot desert in the world.",
    "The Nile River is the longest river in the world."
]

docs = [Document(content=fact) for fact in facts]
doc_store = InMemoryDocumentStore()

In [5]:
pipeline = Pipeline()
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

pipeline.add_component(
    instance=SentenceTransformersDocumentEmbedder(model=embedding_model), 
    name="doc_embedder"
)

pipeline.add_component(
    name="doc_writer",
    instance = DocumentWriter(document_store=doc_store) 
)

pipeline.connect('doc_embedder.documents', 'doc_writer.documents')

pipeline.run({'doc_embedder': {"documents": docs}})



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'doc_writer': {'documents_written': 10}}

# RAG
haystack.components.embedders.SentenceTransformersTextEmbedder
haystack.components.retrievers.in_memory.InMemoryEmbeddingRetriever
haystack.components.builders.ChatPromptBuilder
haystack.dataclasses.ChatMessage
haystack.components.generators.chat.OpenAIChatGenerator

In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

In [7]:
template = [
    ChatMessage.from_system(
    """
    Answer the questions based on the given context.

    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}
    Question: {{ question }}
    Answer:
    """
    )
]

rag = Pipeline()

rag.add_component('embedder', SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
rag.add_component('retriever', InMemoryEmbeddingRetriever(document_store = doc_store))
rag.add_component('prompt_builder', ChatPromptBuilder(template = template))
rag.add_component('llm', OpenAIChatGenerator(model = 'gpt-4o-mini'))

rag.connect('embedder', 'retriever.query_embedding')
rag.connect('retriever', 'prompt_builder.documents')
rag.connect('prompt_builder.prompt', 'llm.messages')

rag.draw("sample.png")


In [8]:
def ask(question):
    return rag.run({'embedder': {'text': question}, 'prompt_builder': {'question' : question} })


res = ask("What is some facts about earth")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
print(res)
print(res['llm']['replies'][0].text)

{'llm': {'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text="Here are some facts about Earth:\n\n1. Earth is the only planet in our solar system not named after a god.\n2. Antarctica is the driest, windiest, and coldest continent on Earth.\n3. The Amazon rainforest produces more than 20% of the world's oxygen supply.\n4. Mount Everest is the highest point on Earth.\n5. The Pacific Ocean is the largest and deepest ocean on Earth.\n6. There are more than 24 time zones around the world.\n7. Russia is the largest country by land area.\n8. The Sahara Desert is the largest hot desert in the world.\n9. The Great Wall of China is the longest man-made structure in the world.\n10. The Nile River is the longest river in the world.")], _name=None, _meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 155, 'prompt_tokens': 182, 'total_tokens': 337, 'completion_tokens_details': CompletionTokensDetails(ac

# Using tools in Haystack

In [10]:
# Pipeline as a tool
from haystack.tools import Tool

params = {
    'type': 'object',
    'properties': {
        'question': {
            'type': 'string',
            'description': 'Query used for search. Infer this information from user message'
        }
    },
    'required': ['question']
}

rag_tool = Tool(
    name='rag_pipeline_tool',
    description="Get information about provided facts in the document_store",
    parameters=params,
    function = ask,
)

In [11]:
#2. Function as a tool
from typing import Annotated, Literal
from haystack.tools import create_tool_from_function

WEATHER_INFO = {
    "Berlin": {"weather": "mostly sunny", "temperature": 7, "unit": "celsius"},
    "Paris": {"weather": "mostly cloudy", "temperature": 8, "unit": "celsius"},
    "Rome": {"weather": "sunny", "temperature": 14, "unit": "celsius"},
    "Madrid": {"weather": "sunny", "temperature": 10, "unit": "celsius"},
    "London": {"weather": "cloudy", "temperature": 9, "unit": "celsius"},
}


def get_weather(
        city: Annotated[str, 'the city for which to get weather'] = 'Berlin',
        unit: Annotated[Literal["Celsius", "Fahrenheit"], 'the temperature unit'] = 'Berlin'):
    '''A simple function to get the current weather for a location'''
    if city in WEATHER_INFO:
        return WEATHER_INFO[city]
    else:
        return {"weather": "sunny", "temperature": 21.8, "unit": "fahrenheit"}

weather_tool = create_tool_from_function(get_weather)


# Running OpenAIChatGenerator with tools

In [12]:
from haystack.dataclasses import ChatMessage
from haystack.components.tools import ToolInvoker


In [13]:
user_messages = [
    ChatMessage.from_system(
        "Use the tool that you're provided with. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    ),
    ChatMessage.from_user('Can you tell me 1 interesting fact')
]

res = chat_gen.run(messages = user_messages, tools = [rag_tool, weather_tool])

In [14]:
pprint(res['replies'][0])

ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>,
            _content=[ToolCall(tool_name='rag_pipeline_tool',
                               arguments={'question': 'Tell me an interesting '
                                                      'fact'},
                               id='call_5FJDCudTaBDZCn1ta1SHAUri')],
            _name=None,
            _meta={'finish_reason': 'tool_calls',
                   'index': 0,
                   'model': 'gpt-4o-mini-2024-07-18',
                   'usage': {'completion_tokens': 21,
                             'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0),
                             'prompt_tokens': 160,
                             'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0),
                             'total_tokens': 181}})


In [15]:
tool_invoker = ToolInvoker(tools = [rag_tool, weather_tool])

tool_res_message = tool_invoker.run(messages = res['replies'])['tool_messages']
# ToolCallResult is a json. so you can call the tool in your code
print(tool_res_message)


/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/haystack/components/tools/tool_invoker.py:123: UserWarning: The `ToolInvoker` component is experimental and its API may change in the future.
  warnings.warn(msg)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[ChatMessage(_role=<ChatRole.TOOL: 'tool'>, _content=[ToolCallResult(result='{\'llm\': {\'replies\': [ChatMessage(_role=<ChatRole.ASSISTANT: \'assistant\'>, _content=[TextContent(text="An interesting fact is that the Amazon rainforest produces more than 20% of the world\'s oxygen supply.")], _name=None, _meta={\'model\': \'gpt-4o-mini-2024-07-18\', \'index\': 0, \'finish_reason\': \'stop\', \'usage\': {\'completion_tokens\': 21, \'prompt_tokens\': 181, \'total_tokens\': 202, \'completion_tokens_details\': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), \'prompt_tokens_details\': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}})]}}', origin=ToolCall(tool_name='rag_pipeline_tool', arguments={'question': 'Tell me an interesting fact'}, id='call_5FJDCudTaBDZCn1ta1SHAUri'), error=False)], _name=None, _meta={})]


In [16]:
print(res['replies'])

final_message = user_messages + res['replies'] + tool_res_message

final_rep = chat_gen.run(messages = final_message, tools = [rag_tool, weather_tool])

[ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[ToolCall(tool_name='rag_pipeline_tool', arguments={'question': 'Tell me an interesting fact'}, id='call_5FJDCudTaBDZCn1ta1SHAUri')], _name=None, _meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {'completion_tokens': 21, 'prompt_tokens': 160, 'total_tokens': 181, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}})]


In [17]:
pprint(final_rep)

{'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>,
                         _content=[TextContent(text='An interesting fact is '
                                                    'that the Amazon '
                                                    'rainforest produces more '
                                                    "than 20% of the world's "
                                                    'oxygen supply.')],
                         _name=None,
                         _meta={'finish_reason': 'stop',
                                'index': 0,
                                'model': 'gpt-4o-mini-2024-07-18',
                                'usage': {'completion_tokens': 22,
                                          'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0),
                                          'prompt_tokens': 354,
                          